# Auto-TS
Automatically build ARIMA, SARIMAX, VAR, FB Prophet and XGBoost Models on Time Series data sets with a Single Line of Code. Now updated with Dask to handle millions of rows.
## Github: [https://github.com/AutoViML/Auto_TS](https://github.com/AutoViML/Auto_TS)

<img src="https://github.com/AutoViML/Auto_TS/raw/master/logo.png" alt="auto-ts"/>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from load_kaggle import load_kaggle

In [ ]:
subm, train, test = load_kaggle()
print(train.shape, test.shape, subm.shape)

In [ ]:
!pip install auto-ts

In [ ]:
!pip install xlrd

In [ ]:
from auto_ts import auto_timeseries

In [ ]:
train.head(2)

In [ ]:
## The data is hourly time series ####
targets = ['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides']
ts_column = 'date_time'

In [ ]:
model = auto_timeseries(score_type='rmse',
                time_interval='H',
                non_seasonal_pdq=None, 
                seasonality=False, seasonal_period=12,
                model_type=['ml'],
                verbose=2)

# Let's build a model using each target while dropping the other two targets

In [ ]:
import copy
all_preds = []
targets_copy = copy.deepcopy(targets)
for i, each_target in enumerate(targets_copy):
    rem_targets = [x for x in targets if x != each_target]
    train1 = train.drop(rem_targets, axis=1)
    model.fit(train1, ts_column, each_target)
    preds1 = model.predict(test, model='best')
    if i == 0:
        all_preds = preds1['yhat'].values
    else:
        all_preds = np.c_[all_preds, preds1['yhat'].values]
print('Completed. ', all_preds.shape)

In [ ]:
subm[targets] = all_preds
subm.head()

In [ ]:
subm.to_csv('submission_1.csv', index=False)